# Understand Number Space - MinMax Mode

To enable searches with well known MAX and MIN logic `NumberSpace` creates the opportunity to embed numeric data in a searchable way.

Check Similar mode [here](number_embedding_similar.ipynb).

In [1]:
%pip install superlinked==17.8.3

In [2]:
from superlinked import framework as sl

In [3]:
class Person(sl.Schema):
    id: sl.IdField
    height: sl.Float  # set height in cms for example
    outstanding_debt_amount: sl.Float  # set in dollars for example


person = Person()

## Maximum mode

To prefer higher numbers. For height, a linear scale is sufficient - which is the default.

In [4]:
height_space = sl.NumberSpace(
    number=person.height, min_value=100, max_value=200, mode=sl.Mode.MAXIMUM
)  # heights out of this range are considered an outlier in our use case

## Minimum mode

For preferring lower numbers. As debt often follows a power law distribution (most people have low debt while a small amount of people have extremely high), a logarithmic scale is useful to better embed numbers. Refer to this [discussion](https://www.reddit.com/r/explainlikeimfive/comments/nk990a/eli5_what_is_a_logarithmic_scale_why_is_it_more/) for example.

In [5]:
debt_space = sl.NumberSpace(
    number=person.outstanding_debt_amount,
    min_value=0,
    max_value=1e6,
    mode=sl.Mode.MINIMUM,
    scale=sl.LogarithmicScale(10),
)  # debts higher than max_value are effectively capped

In [6]:
person_index = sl.Index([height_space, debt_space])

In [7]:
source: sl.InMemorySource = sl.InMemorySource(person)
executor = sl.InMemoryExecutor(sources=[source], indices=[person_index])
app = executor.run()

In [8]:
# let's insert some numbers tailored to create a point.
source.put(
    [
        {
            "id": "person1",
            "height": 190,
            "outstanding_debt_amount": 1e6,
        },
        {
            "id": "person2",
            "height": 150,
            "outstanding_debt_amount": 1e3,  # mediocre in log scale,
        },
        {
            "id": "person3",
            "height": 150,
            "outstanding_debt_amount": 0,
        },
    ]
)

In [9]:
query = sl.Query(person_index, weights={height_space: 1, debt_space: 1}).find(person)

In [10]:
result = app.query(query)
result.to_pandas()

,height,outstanding_debt_amount,id,similarity_score
0,150,0.0,person3,0.603553
1,150,1000.0,person2,0.499972
2,190,1000000.0,person1,0.349201


The following table explains the results. Notice on the log scale, the mediocre number is much lower than `1e6 / 2` - resulting in more differentiating power for the lower values of the range provided by the embedding.

<style>
.table_component {
    overflow: auto;
    width: 100%;
}

.table_component table {
    border: 1px solid #dededf;
    height: 100%;
    width: 100%;
    table-layout: fixed;
    border-collapse: collapse;
    border-spacing: 1px;
    text-align: left;
}

.table_component caption {
    caption-side: top;
    text-align: left;
}

.table_component th {
    border: 1px solid #dededf;
    background-color: #eceff1;
    color: #000000;
    padding: 5px;
}

.table_component td {
    border: 1px solid #dededf;
    background-color: #ffffff;
    color: #000000;
    padding: 5px;
}
</style>
<div class="table_component" role="region" tabindex="0">
<table>
    <caption>
        <p><b>Explanation of results</b></p>
    </caption>
    <thead>
        <tr>
            <th><b>Rank</b></th>
            <th><b>Id</b></th>
            <th><b>Debt status</b></th>
            <th><b>Height</b></th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>1<br></td>
            <td>person3<br></td>
            <td>perfect</td>
            <td>mediocre<br></td>
        </tr>
        <tr>
            <td>2</td>
            <td>person2</td>
            <td>mediocre (log scale)</td>
            <td>mediocre</td>
        </tr>
        <tr>
            <td>3</td>
            <td>person1</td>
            <td>worst possible</td>
            <td>almost! perfect</td>
        </tr>
    </tbody>